In [290]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal

import arviz as az
import xarray as xr

from utils import traceplot, acfplot, pairplot

# from rosenbrock import sample_rosenbrock, logpi_rosenbrock, partial_logpi_rosenbrock, hessian_logpi_rosenbrock
from ipynb.fs.full.rosenbrock import sample_rosenbrock, logpi_rosenbrock, partial_logpi_rosenbrock, hessian_logpi_rosenbrock

In [291]:
def ess(X, n1, n2):
    idx = ['X_{1}']

    for j in range(1, n2+1):
        for i in range(2, n1+1):
            idx.append(f'X_{{{j},{i}}}')

    print('ESS for each component:')

    for i in range(X.shape[0]):
        print('\t', idx[i], ': ', az.ess(X[i, :]))

In [292]:
def compare_quantiles(X_mcmc, n1, n2, n_iter=10000):
    quantiles = (0.025, 0.25, 0.5, 0.75, 0.975)

    idx = ['X_{1}']

    for j in range(1, n2+1):
        for i in range(2, n1+1):
            idx.append(f'X_{{{j},{i}}}')

    X_true = sample_rosenbrock(n_iter, n1, n2)

    df = pd.DataFrame(columns=['0.025', '0.25', '0.5', '0.75', '0.975'])

    for i in range(X_true.shape[0]):
        df.loc['True ' + idx[i]] = np.quantile(X_true[i, :], quantiles)
        df.loc['MCMC ' + idx[i]] = np.quantile(X_mcmc[i, :], quantiles)

    return df


In [293]:
def softabs(lam, alpha=1.0, small=1e-8, large=20.0):
    x = alpha * lam

    # Near zero: lam*coth(alpha*lam) ~ 1/alpha + alpha*lam^2/3 + ...
    if abs(x) < small:
        return 1.0 / alpha
    
    # Large positive: coth(x) ~ 1, so lam*coth(x) ~ lam
    # Large negative: coth(x) ~ -1, so lam*coth(x)
    if abs(x) > large:
        return abs(lam)
    
    # Otherwise, directly compute lam*coth(alpha*lam) = lam*cosh(x)/sinh(x)
    return lam * (np.cosh(x) / np.sinh(x))


def project_to_psd(A, method='clip', epsilon=1e-8, alpha=1e3):
    '''
    Project a matrix to the positive semidefinite cone.
    '''
    eigenvalues, eigenvectors = np.linalg.eigh(A)

    if method == 'clip':
        eigenvalues = np.maximum(eigenvalues, epsilon)
        return eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
    
    elif method == 'abs':
        eigenvalues = np.abs(eigenvalues)
        return eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
    
    elif method == 'softabs':
        eigenvalues_soft = np.array([softabs(eigenvalue, alpha) for eigenvalue in eigenvalues])
        return eigenvectors @ np.diag(np.maximum(eigenvalues_soft, epsilon)) @ eigenvectors.T

    else:
        return A

def is_psd(A, tol=1e-8):
    A = (A + A.T) / 2
    
    eigenvalues = np.linalg.eigvalsh(A)

    # Check if all eigenvalues are non-negative (within tolerance)
    return np.all(eigenvalues >= -tol)


# N1 = 3
# N2 = 2
# X_INIT = [0, 0, 0, 0, 0]
# N_ITER = 100000

# X_smmala, accept_rate = SMMALA(
#     logpi=logpi_rosenbrock,
#     partial_logpi=partial_logpi_rosenbrock,
#     hessian_logpi=hessian_logpi_rosenbrock,
#     n_iter=N_ITER,
#     x_init=X_INIT,
#     step_size=0.18,
#     n1=N1,
#     n2=N2,
#     method='softabs'
# )
# print(f'SMMALA acceptance rate: {accept_rate}')

# traceplot(X_smmala)

# acfplot(X_smmala)

# pairplot(X_smmala, N1, N2)

# MCMC Algorithms


## Random Walk Metropolis


In [294]:
def RWM_proposal(x, step_size):
    return x + step_size * np.random.normal(size=x.shape[0])


def RWM(logpi, n_iter, x_init, step_size=1, **kwargs):
    x = np.asarray(x_init)

    # (#components, #iterations)
    X = np.empty((x.shape[0], n_iter))

    # Counter for accepted proposals
    accepted = 0

    logpi_x = logpi(x, **kwargs)

    for i in range(n_iter):
        # Proposal state
        y = RWM_proposal(x, step_size)

        logpi_y = logpi(y, **kwargs)

        # Log RWM acceptance rate
        log_acceptance = logpi_y - logpi_x

        # Acceptance criterion
        if np.log(np.random.uniform(size=1)) < log_acceptance:
            x = y
            logpi_x = logpi_y
            accepted += 1

        X[:, i] = x

    acceptance_rate = accepted / n_iter

    return X, acceptance_rate

## Metropolis-Adjusted Langevin Algorithm


In [295]:
def MALA_proposal(x, partial_logpi_x, step_size):
    z = step_size * np.random.normal(size=x.shape[0])

    return x + (1/2) * (step_size**2) * partial_logpi_x + z


def MALA_logq_ratio(x, y, partial_logpi_x, partial_logpi_y, step_size):
    log_xy = multivariate_normal.logpdf(y, mean=(x + (1/2) * (step_size**2) * partial_logpi_x), cov=(step_size**2))
    log_yx = multivariate_normal.logpdf(x, mean=(y + (1/2) * (step_size**2) * partial_logpi_y), cov=(step_size**2))
    
    return log_yx - log_xy


def MALA(logpi, partial_logpi, n_iter, x_init, step_size=1, **kwargs):
    x = np.asarray(x_init)

    # (#components, #iterations)
    X = np.empty((x.shape[0], n_iter))

    # Counter for accepted proposals
    accepted = 0

    logpi_x = logpi(x, **kwargs)

    for i in range(n_iter):
        partial_logpi_x = partial_logpi(x, **kwargs)
        
        # Proposal state
        y = MALA_proposal(x, partial_logpi_x, step_size)
        
        logpi_y = logpi(y, **kwargs)
        partial_logpi_y = partial_logpi(y, **kwargs)

        # Log preconditioned MALA acceptance rate
        log_acceptance = logpi_y - logpi_x + MALA_logq_ratio(x, y, partial_logpi_x, partial_logpi_y, step_size)

        # Acceptance criterion
        if np.log(np.random.uniform(size=1)) < log_acceptance:
            x = y
            logpi_x = logpi_y
            accepted += 1

        X[:, i] = x
        
    acceptance_rate = accepted / n_iter

    return X, acceptance_rate

## Barker Metropolis-Hastings


In [296]:
def stable_log1p_exp(x):
    '''
    Compute log(1 + exp(x)) using the log-sum-exp trick for numerical stability.
    '''
    # return np.where(x > 0, x + np.log1p(np.exp(-x)), np.log1p(np.exp(x)))

    # For large positive x, exp(-x) -> 0, so directly return x
    return np.where(x > 700, x, np.where(x > 0, x + np.log1p(np.exp(-x)), np.log1p(np.exp(x))))


def Barker_proposal(x, partial_logpi_x, step_size):
    # Magnitude
    z = step_size * np.random.normal(size=len(x), scale=1)

    # Direction
    threshold = 1 / (1 + np.exp(- z * partial_logpi_x))
    b = np.where(np.random.uniform(size=1) < threshold, 1, -1) 

    return x + b * z


def Barker_logq_ratio(x, y, partial_logpi_x, partial_logpi_y):
    z = y - x

    logq_xy = - np.log1p(np.exp(- z * partial_logpi_x))
    logq_yx = - np.log1p(np.exp(z * partial_logpi_y))

    # logq_xy = -stable_log1p_exp(-z * partial_logpi_x)
    # logq_yx = -stable_log1p_exp(z * partial_logpi_y)

    return np.sum(logq_yx - logq_xy)
   

def Barker(logpi, partial_logpi, n_iter, x_init, step_size=1, **kwargs):
    x = np.asarray(x_init)

    # (#components, #iterations)
    X = np.empty((x.shape[0], n_iter))

    # Counter for accepted proposals
    accepted = 0

    logpi_x = logpi(x, **kwargs)
    
    for i in range(n_iter):
        partial_logpi_x = partial_logpi(x, **kwargs)

        # Proposal state
        y = Barker_proposal(x, partial_logpi_x, step_size)

        logpi_y = logpi(y, **kwargs)
        partial_logpi_y = partial_logpi(y, **kwargs)

        # Log Barker acceptance rate
        log_acceptance = logpi_y - logpi_x + Barker_logq_ratio(x, y, partial_logpi_x, partial_logpi_y)
        
        # Acceptance criterion
        if np.log(np.random.uniform(size=1)) < log_acceptance:
            x = y
            logpi_x = logpi_y
            accepted +=1

        X[:, i] = x
    
    acceptance_rate = accepted / n_iter

    return X, acceptance_rate 

## Simplified Manifold Metropolis-Adjusted Langevin Algorithm


In [297]:
def SMMALA_proposal(x, partial_logpi_x, A_x, step_size, method):
    z = step_size * np.random.normal(size=x.shape[0]) 

    L_x = np.linalg.cholesky(project_to_psd(A_x, method))

    return x + (1/2) * (step_size**2) * A_x @ partial_logpi_x + L_x @ z


def SMMALA_logq_ratio(x, y, partial_logpi_x, partial_logpi_y, A_x, A_y, step_size, method):
    mean_xy = x + (1/2) * (step_size**2) * A_x @ partial_logpi_x
    mean_yx = y + (1/2) * (step_size**2) * A_y @ partial_logpi_y

    cov_xy = (step_size**2) * A_x
    cov_yx = (step_size**2) * A_y

    log_xy = multivariate_normal.logpdf(y, mean=mean_xy, cov=project_to_psd(cov_xy, method))
    log_yx = multivariate_normal.logpdf(x, mean=mean_yx, cov=project_to_psd(cov_yx, method))
    
    return log_yx - log_xy


def SMMALA(logpi, partial_logpi, hessian_logpi, n_iter, x_init, step_size=1, method='clip', **kwargs):
    x = np.asarray(x_init)

    # (#components, #iterations)
    X = np.empty((x.shape[0], n_iter))

    # Counter for accepted proposals
    accepted = 0

    logpi_x = logpi(x, **kwargs)

    for i in range(n_iter):
        partial_logpi_x = partial_logpi(x, **kwargs)
        H_x = hessian_logpi(x, **kwargs)
        A_x = np.linalg.inv(-H_x)

        # Proposal state
        y = SMMALA_proposal(x, partial_logpi_x, A_x, step_size, method)
        
        logpi_y = logpi(y, **kwargs)
        partial_logpi_y = partial_logpi(y, **kwargs)
        H_y = hessian_logpi(y, **kwargs)
        A_y = np.linalg.inv(-H_y)

        # Log SMMALA acceptance rate
        log_acceptance = logpi_y - logpi_x + SMMALA_logq_ratio(x, y, partial_logpi_x, partial_logpi_y, A_x, A_y, step_size, method)

        # Acceptance criterion
        if np.log(np.random.uniform(size=1)) < log_acceptance:
            x = y
            logpi_x = logpi_y
            accepted += 1

        X[:, i] = x
        
    acceptance_rate = accepted / n_iter

    return X, acceptance_rate

In [ ]:
N1 = 3
N2 = 2
X_INIT = [0, 0, 0, 0, 0]
N_ITER = 10000

X_clip, accept_rate = SMMALA(
    logpi=logpi_rosenbrock,
    partial_logpi=partial_logpi_rosenbrock,
    hessian_logpi=hessian_logpi_rosenbrock,
    n_iter=N_ITER,
    x_init=X_INIT,
    step_size=0.3,
    n1=N1,
    n2=N2,
    method='clip'
)
print(f'SMMALA acceptance rate: {accept_rate}')

traceplot(X_clip)
acfplot(X_clip)

ess(X_clip, n1=N1, n2=N2)
compare_quantiles(X_clip, n1=N1, n2=N2)

In [ ]:
X_abs, accept_rate = SMMALA(
    logpi=logpi_rosenbrock,
    partial_logpi=partial_logpi_rosenbrock,
    hessian_logpi=hessian_logpi_rosenbrock,
    n_iter=N_ITER,
    x_init=X_INIT,
    step_size=0.35,
    n1=N1,
    n2=N2,
    method='abs'
)
print(f'SMMALA acceptance rate: {accept_rate}')

traceplot(X_abs)
acfplot(X_abs)

ess(X_abs, n1=N1, n2=N2)
compare_quantiles(X_abs, n1=N1, n2=N2)

In [ ]:
pairplot([X_abs, X_clip], N1, N2, sampler_names=['Abs', 'Clip'])

## Simplified Manifold Barker Metropolis-Hastings


In [302]:
def SMBarker_proposal(x, partial_logpi_x, L_x, step_size=1):
    # Magnitude
    z = step_size * np.random.normal(size=len(x), scale=1)

    # Direction
    threshold = 1 / (1 + np.exp(- z * (partial_logpi_x @ L_x)))
    b = np.where(np.random.uniform(size=1) < threshold, 1, -1) 

    return x + L_x @ (b * z)


def SMBarker_logq_ratio(x, y, partial_logpi_x, partial_logpi_y, L_x, L_y):
    z_xy = np.linalg.inv(L_x) @ (x - y)
    z_yx = np.linalg.inv(L_y) @ (y - x)

    logq_xy = - np.log1p(np.exp(z_xy @ (partial_logpi_x @ L_x)))
    logq_yx = - np.log1p(np.exp(z_yx @ (partial_logpi_y @ L_y)))

    return np.sum(logq_yx - logq_xy)


def SMBarker(logpi, partial_logpi, hessian_logpi, n_iter, x_init, step_size=1, method='clip', **kwargs):
    x = np.asarray(x_init)

    # (#components, #iterations)
    X = np.empty((x.shape[0], n_iter))

    # Counter for accepted proposals
    accepted = 0

    logpi_x = logpi(x, **kwargs)

    for i in range(n_iter):
        partial_logpi_x = partial_logpi(x, **kwargs)
        H_x = hessian_logpi(x, **kwargs)
        A_x = project_to_psd(np.linalg.inv(-H_x), method)
        L_x = np.linalg.cholesky(A_x)

        # Propose candidate state
        y = SMBarker_proposal(x, partial_logpi_x, L_x, step_size)

        logpi_y = logpi(y, **kwargs)
        partial_logpi_y = partial_logpi(y, **kwargs)
        H_y = hessian_logpi(y, **kwargs)
        A_y = project_to_psd(np.linalg.inv(-H_y), method)
        L_y = np.linalg.cholesky(A_y)

        # Log SMBarker acceptance rate
        log_acceptance = logpi_y - logpi_x + SMBarker_logq_ratio(x, y, partial_logpi_x, partial_logpi_y, L_x, L_y)

        # Acceptance criterion
        if np.log(np.random.uniform(size=1)) < log_acceptance:
            x = y
            logpi_x = logpi_y
            accepted += 1

        X[:, i] = x
        
    acceptance_rate = accepted / n_iter

    return X, acceptance_rate


In [ ]:
N1 = 3
N2 = 2
X_INIT = [0, 0, 0, 0, 0]
N_ITER = 10000

X_clip, accept_rate = SMBarker(
    logpi=logpi_rosenbrock,
    partial_logpi=partial_logpi_rosenbrock,
    hessian_logpi=hessian_logpi_rosenbrock,
    n_iter=N_ITER,
    x_init=X_INIT,
    step_size=0.9,
    n1=N1,
    n2=N2,
    method='clip'
)
print(f'SMBarker acceptance rate: {accept_rate}')

traceplot(X_clip)
acfplot(X_clip)

ess(X_clip, n1=N1, n2=N2)
compare_quantiles(X_clip, n1=N1, n2=N2)

In [ ]:
X_abs, accept_rate = SMBarker(
    logpi=logpi_rosenbrock,
    partial_logpi=partial_logpi_rosenbrock,
    hessian_logpi=hessian_logpi_rosenbrock,
    n_iter=N_ITER,
    x_init=X_INIT,
    step_size=0.8,
    n1=N1,
    n2=N2,
    method='abs'
)
print(f'SMBarker acceptance rate: {accept_rate}')

traceplot(X_abs)
acfplot(X_abs)

ess(X_abs, n1=N1, n2=N2)
compare_quantiles(X_abs, n1=N1, n2=N2)

In [ ]:
pairplot([X_abs, X_clip], N1, N2, sampler_names=['Abs', 'Clip'])

# Experiment


Standard parametrization:

-   $\mu = 1$
-   $a  = 1/20$
-   $b_{i, j} = 100/20 \quad (\forall i, j)$


## N1 = 2, N2 = 2


In [ ]:
N1 = 2
N2 = 2
X_INIT = [0, 0, 0]
N_ITER = 100000

X_smmala, accept_rate = SMMALA(
    logpi=logpi_rosenbrock,
    partial_logpi=partial_logpi_rosenbrock,
    hessian_logpi=hessian_logpi_rosenbrock,
    n_iter=N_ITER,
    x_init=X_INIT,
    step_size=0.7,
    n1=N1,
    n2=N2,
    method='abs'
)
print(f'SMMALA acceptance rate: {accept_rate}')

traceplot(X_smmala)
acfplot(X_smmala)
pairplot(X_smmala, N1, N2)

In [ ]:
# X_smbarker, accept_rate = SMBarker(
#     logpi=logpi_rosenbrock,
#     partial_logpi=partial_logpi_rosenbrock,
#     hessian_logpi=hessian_logpi_rosenbrock,
#     n_iter=N_ITER,
#     x_init=X_INIT,
#     step_size=1.15,
#     n1=N1,
#     n2=N2,
# )
# print(f'SMBarker acceptance rate: {accept_rate}')

# traceplot(X_smbarker)
# acfplot(X_smbarker)
# pairplot(X_smbarker, N1, N2)

X_smbarker, accept_rate = SMBarker(
    logpi=logpi_rosenbrock,
    partial_logpi=partial_logpi_rosenbrock,
    hessian_logpi=hessian_logpi_rosenbrock,
    n_iter=N_ITER,
    x_init=X_INIT,
    step_size=1,
    n1=N1,
    n2=N2,
    method='abs'
)
print(f'SMBarker acceptance rate: {accept_rate}')

traceplot(X_smbarker)
acfplot(X_smbarker)
pairplot(X_smbarker, N1, N2)

In [ ]:
pairplot([X_smbarker, X_smmala], n1=N1, n2=N2, sampler_names=['SMBarker', 'SMMALA'])

In [ ]:
pairplot([X_smmala, X_smbarker], n1=N1, n2=N2, sampler_names=['SMMALA', 'SMBarker'])

In [ ]:
X_true = sample_rosenbrock(n_iter=N_ITER, n1=N1, n2=N2)

pairplot([X_true, X_smbarker, X_smmala], n1=N1, n2=N2, sampler_names=['X_true', 'SMBarker', 'SMMALA'])

In [ ]:
print('SMMALA')
ess(X_smmala, n1=N1, n2=N2)
compare_quantiles(X_smmala, n1=N1, n2=N2)

In [ ]:
print('SMBarker')
ess(X_smbarker, n1=N1, n2=N2)
compare_quantiles(X_smbarker, n1=N1, n2=N2)

## N1 = 3, N2 = 2


In [287]:
N1 = 3
N2 = 2
X_INIT = [0, 0, 0, 0, 0]
N_ITER = 100000

In [ ]:
X_smmala, accept_rate = SMMALA(
    logpi=logpi_rosenbrock,
    partial_logpi=partial_logpi_rosenbrock,
    hessian_logpi=hessian_logpi_rosenbrock,
    n_iter=N_ITER,
    x_init=X_INIT,
    step_size=0.28,
    n1=N1,
    n2=N2,
    method='abs'
)
print(f'SMMALA acceptance rate: {accept_rate}')

traceplot(X_smmala)

acfplot(X_smmala)

pairplot(X_smmala, N1, N2)

ess(X_smmala, n1=N1, n2=N2)
compare_quantiles(X_smmala, n1=N1, n2=N2)

In [ ]:
X_smbarker, accept_rate = SMBarker(
    logpi=logpi_rosenbrock,
    partial_logpi=partial_logpi_rosenbrock,
    hessian_logpi=hessian_logpi_rosenbrock,
    n_iter=N_ITER,
    x_init=X_INIT,
    step_size=0.75,
    n1=N1,
    n2=N2,
    method='clip',
)
print(f'SMBarker acceptance rate: {accept_rate}')

traceplot(X_smbarker)

acfplot(X_smbarker)

pairplot(X_smbarker, N1, N2)

ess(X_smbarker, n1=N1, n2=N2)
compare_quantiles(X_smbarker, n1=N1, n2=N2)

In [ ]:
X_true = sample_rosenbrock(n_iter=N_ITER, n1=N1, n2=N2)

pairplot([X_true, X_smbarker, X_smmala], n1=N1, n2=N2, sampler_names=['X_true', 'SMBarker', 'SMMALA'])